# Code for Pulling Lyrics from Genius

In [1]:
from lyricsgenius import Genius
import os
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import sqlite3
conn = sqlite3.connect('../datasets/lyrics.db')
from requests.exceptions import Timeout

In [2]:
genius = Genius(os.getenv('GENIUS_KEY'))
genius.timeout = 15
genius.remove_section_headers = True

In [3]:
full_df = pd.read_csv("../datasets/Database to calculate popularity.csv")


In [9]:
lyric_df = pd.read_sql_query("SELECT * FROM lyrics", conn)


In [10]:
holder_df = full_df[full_df['country']=='USA']
#remove nan and remove uri that are already in lyric_df
holder_df = holder_df[~holder_df['uri'].isin(lyric_df['uri'])]
#remove nan in holder_df
holder_df = holder_df.dropna()
#remove duplicate uri in holder_df
holder_df = holder_df.drop_duplicates(subset='uri')
print(len(holder_df['uri'].unique()))
print(len(holder_df))
holder_df.head(10)


43
43


,Unnamed: 0,country,date,position,uri,track,title,artist
281193,281193,USA,01/11/2020,194.0,https://open.spotify.com/track/1TfqLAPs4K3s2rJ...,"Sweet Dreams - Remastered , Eurythmics, Anni...",Sweet Dreams - Remastered,"Eurythmics, Annie Lennox, Dave Stewart"
281302,281302,USA,31/10/2020,103.0,https://open.spotify.com/track/0kwuKfWntoGh0EW...,"Stranger Things , Kyle Dixon & Michael Stein",Stranger Things,Kyle Dixon & Michael Stein
285389,285389,USA,11/10/2020,190.0,https://open.spotify.com/track/31iqpzg1ZXLLgq0...,"Throat Ba, , BRS Kash",Throat Ba,", BRS Kash"
288794,288794,USA,24/09/2020,195.0,https://open.spotify.com/track/5FmvaZGd6fulojS...,"One Beer , HIXTAPE",One Beer,HIXTAPE
289328,289328,USA,21/09/2020,129.0,https://open.spotify.com/track/5CcPj2miBzqHAxm...,"Thinkin of You , Chan Se Park",Thinkin of You,Chan Se Park
289329,289329,USA,21/09/2020,130.0,https://open.spotify.com/track/1V8bPfh3pgdsHU5...,"Blurred Lines , Oda Kirkwood",Blurred Lines,Oda Kirkwood
289340,289340,USA,21/09/2020,141.0,https://open.spotify.com/track/38HY8JklUp2o1hc...,"Miser , WAVYVIBE",Miser,WAVYVIBE
289344,289344,USA,21/09/2020,145.0,https://open.spotify.com/track/7icOlPlKJ3JSuZU...,"Love Is So Sweet , Missing Seoul",Love Is So Sweet,Missing Seoul
289435,289435,USA,20/09/2020,36.0,https://open.spotify.com/track/6F36SoTLd7pky1n...,"Longing Heart , Mayar Jenisk",Longing Heart,Mayar Jenisk
289674,289674,USA,19/09/2020,75.0,https://open.spotify.com/track/7nz7N7kYxYAir3m...,"Without You , Jennifer Ormond",Without You,Jennifer Ormond


In [53]:

def storeLyrics(songName,artistName,uri,cursor):
    artist = None
    for artist_i in artistName.split('&'):
        artist = genius.search_artist(artist_i, max_songs=0)
        if artist != None:
            break
    if artist==None:
        return None
    song = None
    for song_i in songName.split('-'):
        song = genius.search_song(song_i, artist.name)
        if song != None:
            break
    if song==None:
        return None
    lyrics = song.lyrics.replace('(','').replace(')','').replace('\n', ' ').replace('\r', '').replace('\t', ' ').replace('  ', ' ').replace('\'', '').replace('\'', '\"')
    lyrics = lyrics.split(songName+' Lyrics ')[1]
    query = 'INSERT INTO lyrics(song,artist,lyric,uri) VALUES (\"'+str(songName).replace('\"','\'')+'\",\"'+str(artistName.split(',')).replace('\"','\'')+'\",\"'+lyrics.replace('\"','\'')+'\",\''+str(uri).replace('\"','\'')+'\')'
    cursor.execute(query)
    conn.commit()
def test_store(songName,artistName):
    artist = genius.search_artist(artistName, max_songs=0)
    if artist==None:
        return None
    song = genius.search_song(songName, artist.name)
    if song==None:
        return None
    lyrics = song.lyrics.replace('(','').replace(')','')

In [5]:
cur = conn.cursor()
try:
    cur.execute("CREATE TABLE lyrics (song text NOT NULL, artist text NOT NULL, lyric text NOT NULL, uri text NOT NULL, sentiment Integer NOT NULL, polarity Integer NOT NULL, subjectivity Integer NOT NULL, vader Integer NOT NULL, vader_pos Integer NOT NULL, vader_neg Integer NOT NULL, vader_neut Integer NOT NULL);")
except:
    print("Table already exists")
cur.execute("ALTER TABLE lyrics ADD COLUMN vader_neut Integer;")

In [2]:
taken = []
for index,row in holder_df[::-1].iterrows():
    retries = 0
    if (row['uri'] in taken):
        pass
    else:
        while retries<3:
            try:
                storeLyrics(row['title'],row['artist'],row['uri'],cur)
                taken.append(row['uri'])
                retries = 3
            except Timeout as e:
                retries += 1
                continue

conn.close()

"for index,row in holder_df[::-1].iterrows():\n    retries = 0\n    if (row['uri'] in taken):\n        pass\n    else:\n        while retries<3:\n            try:\n                storeLyrics(row['title'],row['artist'],row['uri'],cur)\n                taken.append(row['uri'])\n                retries = 3\n            except Timeout as e:\n                retries += 1\n                continue\n"

In [ ]:
conn.commit()
conn.close()